# Project 1  
### The Moonshadow Elves  
* Hongmei Zeng  
* Noah Stettler  
* Patrick Harrington  
* Yacub Bholat  
  
Due: 13 January 2020  

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import requests

In [14]:
path = os.path.join('Data', 'SeriesReport-20200112162716_5da8f4.csv')
df = pd.read_csv(path)
df.head(20)

,Local Area Unemployment Statistics,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Original Data Value,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,Series Id:,"LAUCT480500000000003,LAUCT480500000000004,LAUC...",NaN,NaN,NaN,NaN
3,Not Seasonally Adjusted,NaN,NaN,NaN,NaN,NaN
4,Area:,"Austin city, TX",NaN,NaN,NaN,NaN
5,Area Type:,"Cities and towns above 25,000 population",NaN,NaN,NaN,NaN
6,State/Region/Division:,Texas,NaN,NaN,NaN,NaN
7,Years:,2010 to 2019,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN
9,Year,Period,labor force,employment,unemployment,unemployment rate


In [15]:
cleaned_df = df.drop([0,1,2,3,4,5,6,7,8,9])
cleaned_df.head()

,Local Area Unemployment Statistics,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
10,2010,Jan,459941,429426,30515,6.6
11,2010,Feb,461604,432263,29341,6.4
12,2010,Mar,464429,435535,28894,6.2
13,2010,Apr,470021,441713,28308,6.0
14,2010,May,467577,439559,28018,6.0


In [6]:
names = {'Local Area Unemployment Statistics': 'Year',
         'Unnamed: 1': 'Month',
         'Unnamed: 2': 'Labor Force',
         'Unnamed: 3': 'Employment',
         'Unnamed: 4': 'Unemployment',
         'Unnamed: 5': 'Unemployment Rate %' }
cleaned_df = cleaned_df.rename(columns=names)
cleaned_df = cleaned_df.reset_index(drop=True)
cleaned_df.head(10)

,Year,Month,Labor Force,Employment,Unemployment,Unemployment Rate %
0,2010,Jan,459941,429426,30515,6.6
1,2010,Feb,461604,432263,29341,6.4
2,2010,Mar,464429,435535,28894,6.2
3,2010,Apr,470021,441713,28308,6.0
4,2010,May,467577,439559,28018,6.0
5,2010,Jun,469417,439255,30162,6.4
6,2010,Jul,469724,439522,30202,6.4
7,2010,Aug,470685,441294,29391,6.2
8,2010,Sep,471843,443168,28675,6.1
9,2010,Oct,474470,446328,28142,5.9


In [11]:
cleaned_df['Employment Rate %'] = cleaned_df.apply(lambda df: (int(df['Employment']) / int(df['Labor Force']) * 100), axis=1).round(1) 
cleaned_df.head(10)

,Year,Month,Labor Force,Employment,Unemployment,Unemployment Rate %,Employment Rate %
0,2010,Jan,459941,429426,30515,6.6,93.4
1,2010,Feb,461604,432263,29341,6.4,93.6
2,2010,Mar,464429,435535,28894,6.2,93.8
3,2010,Apr,470021,441713,28308,6.0,94.0
4,2010,May,467577,439559,28018,6.0,94.0
5,2010,Jun,469417,439255,30162,6.4,93.6
6,2010,Jul,469724,439522,30202,6.4,93.6
7,2010,Aug,470685,441294,29391,6.2,93.8
8,2010,Sep,471843,443168,28675,6.1,93.9
9,2010,Oct,474470,446328,28142,5.9,94.1


In [13]:
new_df = cleaned_df[['Year','Month','Labor Force','Employment','Employment Rate %','Unemployment','Unemployment Rate %']]
new_df = new_df[new_df.Year != '2019']
new_df.head(20)

,Year,Month,Labor Force,Employment,Employment Rate %,Unemployment,Unemployment Rate %
0,2010,Jan,459941,429426,93.4,30515,6.6
1,2010,Feb,461604,432263,93.6,29341,6.4
2,2010,Mar,464429,435535,93.8,28894,6.2
3,2010,Apr,470021,441713,94.0,28308,6.0
4,2010,May,467577,439559,94.0,28018,6.0
5,2010,Jun,469417,439255,93.6,30162,6.4
6,2010,Jul,469724,439522,93.6,30202,6.4
7,2010,Aug,470685,441294,93.8,29391,6.2
8,2010,Sep,471843,443168,93.9,28675,6.1
9,2010,Oct,474470,446328,94.1,28142,5.9


In [ ]:
colors = ['red', 'blue', 'green', 'orange']
markers = ['^', 'o', 's', '*']
x = new_df.index
    
for column, i in zip(new_df.columns, range(len(new_df.columns))):
    y = new_df[column]
    plt.errorbar(x, y, yerr=0.2, elinewidth=0.8, 
                 color=colors[i], marker=markers[i], 
                 linestyle=':', linewidth=1,
                 errorevery=1, label=column)
    
plt.xlabel('Treatment Duration(Days)')
plt.ylabel('Met. Sites')
plt.title('Metastatic Spread During Treatment', fontsize=15)
plt.legend()
plt.xlim(-5, max(x) + 2)
plt.ylim(-0.2, max(y) + 0.5)
plt.grid(b=True, axis='y')
plt.figure(figsize=(10,6))
plt.savefig('Fig2 Meta Spread.png')
plt.show()